In [203]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [204]:
full_data = []
num_of_data = 1000


In [205]:
def too_much(table, offset=0):
    for i, data in enumerate(table.find_all('tr')[1:], 1):
        temp = data.text.split('$')
        temp[0] = temp[0].replace(str(i + offset), '')
        temp = [i + offset] + temp

        temp[3] = temp[3].replace('<', '')

        if len(temp) == 4:
            temp[2] = temp[2].replace('-', '')
            temp.insert(3, '0,00000%')

        index = len(temp[3]) - temp[3][::-1].find(',') + 2
        temp.insert(3, temp[3][:index + 1])
        temp.insert(4, temp[4][index + 1:])
        temp.pop(5)
        temp.append(temp[-1][-4:])
        temp[5] = temp[5][:-4]

        index2 = len(temp[5]) - temp[5][::-1].find(',') + 2
        temp.insert(5, temp[5][:index2 + 1])
        temp.insert(6, temp[6][index2 + 1:])
        temp.pop(7)

        full_data.append(temp)


In [206]:
text = ''

for j in range(0, num_of_data + 1, 200):
    with open(f'data/worldwide_gross{j}.txt', 'r', encoding='utf-8') as file:
        text += file.read()


In [207]:
soup = BeautifulSoup(text, 'html.parser')


In [208]:
table = soup.find_all('table')

for j in range(num_of_data // 200):
    too_much(table[j], 200*j)


In [209]:
df = pd.DataFrame(full_data)
df.columns = ['Rank', 'Title', 'Worldwide_gross', 'Domestic_gross', 'Domestic %', 'Foreign_gross', 'Foreign %', 'Year']
df


,Rank,Title,Worldwide_gross,Domestic_gross,Domestic %,Foreign_gross,Foreign %,Year
0,1,Avatar,"2,923,710,708","785,221,649",26.9%,"2,138,489,059",73.1%,2009
1,2,Avengers: Endgame,"2,799,439,100","858,373,000",30.7%,"1,941,066,100",69.3%,2019
2,3,Avatar: The Way of Water,"2,320,250,281","684,075,767",29.5%,"1,636,174,514",70.5%,2022
3,4,Titanic,"2,264,812,968","674,354,882",29.8%,"1,590,458,086",70.2%,1997
4,5,Star Wars: Episode VII - The Force Awakens,"2,071,310,218","936,662,225",45.2%,"1,134,647,993",54.8%,2015
...,...,...,...,...,...,...,...,...
995,996,Weathering with You,"192,352,980","8,056,636",4.2%,"184,296,344",95.8%,2019
996,997,The Equalizer,"192,330,738","101,530,738",52.8%,"90,800,000",47.2%,2014
997,998,Sheep Without a Shepherd,"191,602,146","0,000",00%,"191,602,146",100%,2019
998,999,The Addams Family,"191,502,426","113,502,426",59.3%,"78,000,000",40.7%,1991


In [210]:
df.Worldwide_gross = df.Worldwide_gross.str.replace(',', '')
df.Worldwide_gross = df.Worldwide_gross.astype(np.int64)

df.Domestic_gross = df.Domestic_gross.str.replace(',', '')
df.Domestic_gross = df.Domestic_gross.astype(np.int64)

df['Domestic %'] = df['Domestic %'].str.replace('-', '0')
df['Domestic %'] = df['Domestic %'].str.replace('%', '').astype(float)

df.Foreign_gross = df.Foreign_gross.str.replace(',', '')
df.Foreign_gross = df.Foreign_gross.astype(np.int64)

df['Foreign %'] = df['Foreign %'].str.replace('%', '').astype(float)

df.Year = df.Year.astype(int)


In [211]:
df.to_excel('data.xlsx', index=False)
